In [1]:
using Pkg: @pkg_str
pkg"activate ."

In [2]:
using MLDatasets
using MLDataUtils
using Statistics
using LIBSVM
using DecisionTree
using Flux
using TensorFlow

loaded


# FashionMNIST
https://github.com/zalandoresearch/fashion-mnist

![](https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/doc/img/fashion-mnist-sprite.png)

| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |


In [3]:
function appropriately_dressed(clothing_items...)
    3 ∈ clothing_items && return true  # A dress is both top and bottoms
    
    # need trousers and a top of some kind
    return 1 ∈ clothing_items && length(intersect(clothing_items,(0, 2, 4, 6))) > 0 
end

appropriately_dressed (generic function with 1 method)

In [4]:
function generate_pants_dataset(data, max_obs=Inf)
    combination_images = Vector{Vector{Float32}}()
    is_dressed_labels = Vector{Bool}()
    for (img1, lbl1) in eachobs(shuffleobs(data)), (img2, lbl2) in eachobs(shuffleobs(data))
        push!(combination_images, [img1[:]; img2[:]])
        push!(is_dressed_labels,  appropriately_dressed(lbl1, lbl2))
        if length(is_dressed_labels) >= max_obs
            break
        end
    end
    return combination_images, is_dressed_labels
end

generate_pants_dataset (generic function with 2 methods)

In [5]:
using Random
Random.seed!(4)

train_data = generate_pants_dataset((FashionMNIST.traintensor(), FashionMNIST.trainlabels()), 40_000)
@show mean(last, eachobs(train_data))

test_data = generate_pants_dataset((FashionMNIST.testtensor(), FashionMNIST.testlabels()), 1_000);
@show mean(last, eachobs(test_data))

mean(last, eachobs(train_data)) = 0.10015
mean(last, eachobs(test_data)) = 0.483


0.483

# Construct a Balanced Training Set

In [6]:
balanced_train_data = undersample(last, train_data);

@show nobs(balanced_train_data)
@show mean(last, eachobs(balanced_train_data));

nobs(balanced_train_data) = 8012
mean(last, eachobs(balanced_train_data)) = 0.5


# DecisionTree

In [7]:
function demo_decision_tree(train_data, test_data)
    train_features = collect(reduce(hcat, train_data[1])')
    train_labels = collect(train_data[2])

    # train full-tree classifier
    model = build_tree(train_labels, train_features)
    # prune tree: merge leaves having >= 90% combined purity (default: 100%)
    model = prune_tree(model, 0.9)


    test_features = collect(reduce(hcat, test_data[1])')
    test_labels = collect(test_data[2]);
    classes = apply_tree(model, test_features)
    acc = mean(test_labels .== classes)
    @show acc
end

demo_decision_tree (generic function with 1 method)

In [8]:
@time demo_decision_tree(train_data, test_data)

acc = 0.607
 14.255323 seconds (7.81 M allocations: 911.184 MiB, 4.10% gc time)


0.607

In [9]:
@time demo_decision_tree(balanced_train_data, test_data)

acc = 0.655
  6.563638 seconds (6.38 M allocations: 488.947 MiB, 2.99% gc time)


0.655

## RandomForest

In [10]:
function demo_random_forest(train_data, test_data)
    train_features = collect(reduce(hcat, train_data[1])')
    train_labels = collect(train_data[2])

    model = build_forest(train_labels, train_features)

    test_features = collect(reduce(hcat, test_data[1])')
    test_labels = collect(test_data[2]);
    classes = apply_forest(model, test_features)
    acc = mean(test_labels .== classes)
    @show acc
end

demo_random_forest (generic function with 1 method)

In [11]:
@time demo_random_forest(train_data, test_data)

acc = 0.604
  9.217185 seconds (5.07 M allocations: 2.468 GiB, 10.25% gc time)


0.604

In [12]:
@time demo_random_forest(balanced_train_data, test_data)

acc = 0.652
  1.277041 seconds (133.66 k allocations: 480.575 MiB, 18.36% gc time)


0.652

# LibSVM

In [13]:
function demo_libsvm(train_data, test_data)
    train_features = reduce(hcat, train_data[1])
    train_labels = train_data[2]

    model = svmtrain(train_features, train_labels)

    test_features = reduce(hcat, test_data[1])
    test_labels = test_data[2];

    classes, probs = svmpredict(model, test_features)
    acc = mean(test_labels .== classes)
    @show acc
end



demo_libsvm (generic function with 1 method)

In [15]:
@time demo_libsvm(train_data, test_data)

acc = 0.572
1184.465968 seconds (628.59 k allocations: 1.362 GiB, 0.03% gc time)


0.572

In [14]:
@time demo_libsvm(balanced_train_data, test_data)

acc = 0.619
120.130859 seconds (4.14 M allocations: 553.918 MiB, 0.21% gc time)


0.619

## TensorFlow

In [16]:
function tensorflow_model(in_size = 1568)
    sess = Session(Graph())

    leaky_relu6(x) = 0.01x + nn.relu6(x)

    # Network Definition
    @tf begin
        X = placeholder(Float32, shape=[-1, in_size])

        # Network parameters
        hl_sizes = [512, 128, 64]

        Zs = [X]
        for (ii, hlsize) in enumerate(hl_sizes)
            Wii = get_variable("W_$ii", [get_shape(Zs[end], 2), hlsize], Float32)
            bii = get_variable("b_$ii", [hlsize], Float32)
            Zii = leaky_relu6(Zs[end]*Wii + bii)
            push!(Zs, Zii)
        end

        Wout = get_variable([get_shape(Zs[end], 2), 1], Float32)
        bout = get_variable([1], Float32)
        Y_mat = nn.sigmoid(Zs[end]*Wout + bout)

        Y_pred = dropdims(Y_mat; dims=[2]) #drop first dimention
        
        Y_target = placeholder(Float32, shape=[-1])
        loss = nn.sigmoid_cross_entropy_with_logits(;logits=Y_pred, targets=Y_target)
        mean_loss = mean(loss)
        optimizer = TensorFlow.train.minimize(train.AdamOptimizer(), loss)
    end 
    run(sess, global_variables_initializer())

    return (sess, optimizer)
end

function demo_tf(train_data, test_data)
    sess, opt = tensorflow_model()
    
    train_features = collect(reduce(hcat, train_data[1])')
    train_labels = collect(train_data[2])
    
    for epoch in 1:50
        run(sess, opt, Dict(
            sess.graph["X"] => train_features,
            sess.graph["Y_target"] => train_labels
        ))
    end

    test_features = collect(reduce(hcat, test_data[1])')
    test_labels = collect(test_data[2]);
    
    classes = run(sess, sess.graph["Y_pred"] > 0.5, Dict(
            sess.graph["X"] => test_features
        ))
    @show mean(classes)
    acc = mean(test_labels .== classes)
    @show acc
end



demo_tf (generic function with 1 method)

In [17]:
@time demo_tf(train_data, test_data)

2019-02-09 21:46:12.973510: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.2 AVX AVX2 FMA


mean(classes) = 0.0
acc = 0.517
184.857120 seconds (86.67 M allocations: 17.063 GiB, 2.05% gc time)


0.517

In [18]:
@time demo_tf(balanced_train_data, test_data)

mean(classes) = 0.241
acc = 0.546
 26.454692 seconds (7.71 M allocations: 2.953 GiB, 2.36% gc time)


0.546

# Flux

In [19]:
function flux_model(in_size = 1568)
    leaky_relu6(x) = 0.01x + clamp(x, 0, 6)

    return Chain(
        Dense(in_size, 512, leaky_relu6), 
        Dense(512, 128, leaky_relu6),
        Dense(128, 64, leaky_relu6), 
        Dense(64, 1, σ),
        first
    )
    
end

function demo_flux(train_data, test_data)
    mdl = flux_model()
    
    Flux.train!(
        params(mdl),
        Iterators.repeated(train_data, 50), # 50 epochs
        Flux.ADAM()
    ) do xs, ys # This block repressents 1 epoch
        
        #@show typeof(xs)
        #@show typeof(ys)
        #== 
        mean(eachobs((xs,ys))) do (x,y)
            ŷ = mdl(xs)
            @show typeof(ŷ)
            @show size(ŷ)
            @show typeof(y)
            @show size(y)
            
            #Flux.binarycrossentropy(Float32(y), ŷ; ϵ=eps(Float32))
        end
        ==#
        mean(Flux.binarycrossentropy.(Float32.(ys), map(mdl, xs)))
    end
    
    test_features, test_labels = test_data
    
    probs = mapreduce(Flux.data∘mdl, vcat, test_features)
    classes = probs .> 0
    @show mean(classes)
    acc = mean(test_labels .== classes)
    @show acc
end

demo_flux (generic function with 1 method)

In [ ]:
@time demo_flux(train_data, test_data)

In [ ]:
@time demo_flux(balanced_train_data, test_data)